In [1]:
import glob
from PIL import Image
import cv2 
import numpy as np
import types
import os
import nbutils
#from nbutils import showarray
from matplotlib import pyplot as plt
import scipy
import skimage
#img_ref = lena()[140:,120:][:256,:256] / 255.0
img_ref = cv2.imread("lena.jpg")
def make_noisy(img):
    # add gaussian noise
    img = img + 0.1 * np.random.normal(size=img.shape)
    # add some outliers in on the right side of the image
    m = np.random.rand(*img.shape) < 0.2
    m[:,:160] = 0
    img[m] = np.random.rand(m.sum())
    return img

img_obs = make_noisy(img_ref)

plt.imshow(img_ref)
#Mat mDist32S = new Mat();
skimage.img_as_float(img_obs)
plt.show()
plt.imshow(img_obs)

print(img_obs)
plt.show()

<Figure size 640x480 with 1 Axes>

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


[[[1.04884398e+02 1.27956331e+02 2.19910757e+02]
  [1.04963525e+02 1.28003402e+02 2.19907684e+02]
  [1.05997583e+02 1.26959376e+02 2.18968620e+02]
  ...
  [6.98474367e-02 1.83301109e-01 1.90914443e+02]
  [8.30976437e+01 3.37964646e-01 1.94009942e+02]
  [8.79314802e+01 9.68519340e+01 2.01224509e+02]]

 [[1.04966552e+02 1.28138631e+02 2.20027572e+02]
  [1.04998975e+02 1.27879102e+02 2.19916071e+02]
  [1.06028204e+02 1.26914722e+02 2.19171104e+02]
  ...
  [9.89489825e+01 9.79684518e+01 2.00987189e+02]
  [9.30200001e+01 9.49092476e+01 7.13160703e-01]
  [8.40906891e+01 8.79339467e+01 1.89895033e+02]]

 [[1.03845791e+02 1.27089143e+02 2.19018541e+02]
  [1.04034781e+02 1.27058174e+02 2.18947440e+02]
  [1.05821574e+02 1.26982692e+02 2.19070435e+02]
  ...
  [1.06890250e+02 1.03125405e+02 1.98947083e+02]
  [2.82633856e-01 9.10944371e+01 3.46491392e-01]
  [8.30916554e+01 2.47532378e-01 1.73137896e+02]]

 ...

 [[7.60565816e+01 4.89649465e+01 1.12062725e+02]
  [8.40052316e+01 5.80089828e+01 1.1795

<Figure size 640x480 with 1 Axes>

In [10]:
import glob
from PIL import Image
import cv2 
import numpy as np
import types
import os
#from nbutils import showarray
from matplotlib import pyplot as plt
import scipy
import skimage
  
#img_ref = lena()[140:,120:][:256,:256] / 255.0
img_ref = cv2.imread("smiley.jpg")

def nablaT(G):
    h, w = G.shape[:2]
    I = np.zeros((h, w), G.dtype)
    # note that we just reversed left and right sides
    # of each line to obtain the transposed operator
    I[:, :-1] -= G[:, :-1, 0]
    I[:, 1: ] += G[:, :-1, 0]
    I[:-1]    -= G[:-1, :, 1]
    I[1: ]    += G[:-1, :, 1]
    return I

def nabla(I):
    h, w = I.shape
    G = np.zeros((h, w, 2), I.dtype)
    G[:, :-1, 0] -= I[:, :-1]
    G[:, :-1, 0] += I[:, 1:]
    G[:-1, :, 1] -= I[:-1]
    G[:-1, :, 1] += I[1:]
    return G

def anorm2(a):
    return (a*a).sum(-1)
def anorm(a):
    return np.sqrt( anorm2(a) )

def project_nd(P, r):
    '''perform a pixel-wise projection onto R-radius balls'''
    nP = np.maximum(1.0, anorm(P)/r)
    return P / nP[...,np.newaxis]

def solve_TVL1_multy(imgs, clambda, iter_n=101):
    # setting step sizes and other params
    L2 = 8.0
    tau = 0.02
    sigma = 1.0 / (L2*tau)
    theta = 1.0

    #imgs = np.array(imgs)
    X = imgs[0].copy()
    #X = imgs[0].copy()
    P = nabla(X)
    Rs = np.zeros_like(imgs)
    for i in range(iter_n):
        P = project_nd( P + sigma*nabla(X), 1.0 )
        Rs = np.clip(Rs + sigma*(X-imgs), -clambda, clambda)
        X1 = X - tau*(nablaT(P) + Rs.sum(0))
        X = X1 + theta * (X1 - X)
        if i % 10 == 0:
            print("."),
    #print
    return X

#print("img_ref", img_ref)
#print("img_ref", img_ref)
imgO = solve_TVL1_multy(img_ref, 0.5)
print("imgO",imgO)
plt.imshow(imgO)


plt.show()

.


IndexError: index 10 is out of bounds for axis 0 with size 10

In [3]:
import glob
from PIL import Image
import cv2 
import numpy as np
import types
import os
#from nbutils import showarray
from matplotlib import pyplot as plt
import scipy
import skimage
  
#img_ref = lena()[140:,120:][:256,:256] / 255.0
img_ref = cv2.imread("lena.jpg")

def nablaT(G):
    h, w = G.shape[:2]
    I = np.zeros((h, w), G.dtype)
    # note that we just reversed left and right sides
    # of each line to obtain the transposed operator
    I[:, :-1] -= G[:, :-1, 0]
    I[:, 1: ] += G[:, :-1, 0]
    I[:-1]    -= G[:-1, :, 1]
    I[1: ]    += G[:-1, :, 1]
    return I

def nabla(I):
    h, w = I.shape
    G = np.zeros((h, w, 2), I.dtype)
    G[:, :-1, 0] -= I[:, :-1]
    G[:, :-1, 0] += I[:, 1:]
    G[:-1, :, 1] -= I[:-1]
    G[:-1, :, 1] += I[1:]
    return G

def anorm2(a):
    return (a*a).sum(-1)
def anorm(a):
    return np.sqrt( anorm2(a) )

def project_nd(P, r):
    '''perform a pixel-wise projection onto R-radius balls'''
    nP = np.maximum(1.0, anorm(P)/r)
    return P / nP[...,np.newaxis]

def solve_TVL1_multy(imgs, clambda, iter_n=101):
    # setting step sizes and other params
    L2 = 8.0
    tau = 0.02
    sigma = 1.0 / (L2*tau)
    theta = 1.0
    height, width, channels = imgs.shape
    print("    height, width, channels",    height, width, channels)
    #new = imgs.copy()
    new = np.zeros((256,3))
    #imgs = np.array(imgs)
    #inputIm = np.asarray(inputIm)
    imArr  = np.asarray(imgs)
    for y in range(0, height):
        print("Y IN LOOP IS ", y)

        X = imArr[y].copy()
        #X = imgs[0].copy()
        P = nabla(X)
        Rs = np.zeros_like(imgs)
            #for i in range(iter_n):
        for x in range(0, width):
            P = project_nd( P + sigma*nabla(X), 1.0 )
            Rs = np.clip(Rs + sigma*(X-imgs), -clambda, clambda)
            X1 = X - tau*(nablaT(P) + Rs.sum(0))
            X = X1 + theta * (X1 - X)
            X = np.round(X)
            X = np.asarray(X)
            #print("XTYPE", type(X))
            X.astype(int)
            if x % 10 == 0:
                print("."),
        new[y] = X
        print("X", X)
        print("new[y]", new[y])
        print("imgs[y]", imgs[y])
    return new

#print("img_ref", img_ref)
#print("img_ref", img_ref)
imgO = solve_TVL1_multy(img_ref, 0.5)
#print("imgO",imgO)
plt.imshow(imgO)
plt.show()
plt.imshow(img_ref)
plt.show()

    height, width, channels 256 256 3
Y IN LOOP IS  0
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.


ValueError: could not broadcast input array from shape (256,3) into shape (3)